Count number of classes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# unzip /content/drive/MyDrive/CPSC 599/Assignment 3 Dataset.zip
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/CPSC 599/Assignment 3 Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./')


In [ ]:
import pandas as pd

# Assuming your CSV file has columns 'id' and 'label'
csv_file_path = 'train.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Count the number of images for each label
label_counts = df['label'].value_counts()

# Count the number of unique labels
unique_labels_count = len(label_counts)

# Calculate the average number of images per label
average_images_per_label = label_counts.mean()

# Display the results
print(f"Number of unique labels: {unique_labels_count}")
print("\nNumber of images for each label:")
print(label_counts)
print(f"\nAverage number of images per label: {average_images_per_label:.2f}")


Number of unique labels: 100

Number of images for each label:
68    49
45    47
32    46
84    46
9     46
      ..
89    34
15    34
34    33
53    30
88    28
Name: label, Length: 100, dtype: int64

Average number of images per label: 41.35


Custom Dataset Builder

In [ ]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx, 0]
        img_path = f'{img_name}'
        image = Image.open(img_path)
        label = int(self.data_frame.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

# # Example usage:
# transform = transforms.Compose([
#     #transforms.Resize((299, 299)),  # Assuming Inception input size
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and split dataset
full_dataset = CustomDataset(csv_file='train.csv', root_dir='', transform=transform)
train_size = 0.8
train_dataset, validation_dataset = train_test_split(full_dataset, train_size=train_size, random_state=42)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)





In [ ]:
import torch

# Function to calculate accuracy
def calculate_metrics(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    accuracy = correct / labels.size(0)

    # Return the metric we're optimizing for
    return accuracy

Inception Model Download

In [ ]:
from torchvision import models
import torch.optim as optim
import torch.nn as nn
import torch

# Load the pre-trained Inception model
inception_model = models.inception_v3(pretrained=True)

# Modify the last classification layer
in_features = inception_model.fc.in_features
inception_model.fc = nn.Linear(in_features, 100)  # Assuming 100 classes

# You might want to move your model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception_model = inception_model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:06<00:00, 16.2MB/s]


Inception Model

In [ ]:
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
import torch.autograd.profiler as profiler

# Choose your learning rate and weight decay parameters
learning_rate = 0.001
weight_decay = 1e-5

# Choose the number of epochs
num_epochs = 20

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(inception_model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_accuracy = 0.0

    # Enable profiler for the entire epoch
    with profiler.profile(record_shapes=True) as prof:
        with profiler.record_function("train_epoch"):
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs, aux_outputs = inception_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                scheduler.step()

                # Update metrics
                epoch_loss += loss.item()
                epoch_accuracy += calculate_metrics(outputs, labels)

    # Print and save metrics after each epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Learning Rate: {scheduler.get_last_lr()}')
    print(f'  Loss: {epoch_loss / len(train_loader)}')
    print(f'  Training Accuracy: {epoch_accuracy / len(train_loader)}')

    # Save model parameters
    torch.save(inception_model.state_dict(), f'model_epoch_{epoch + 1}.pth')

# Use the profiler results
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


Resnet Model

In [ ]:
from torchvision import datasets, transforms, models
from torchvision.models import ResNet18_Weights, resnet18
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Load pre-trained ResNet model
weights = ResNet18_Weights.IMAGENET1K_V1
resnet = resnet18(weights=weights)

# Modify the final fully connected layer
num_classes = 100
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training loop with early stopping and learning rate scheduler
num_epochs = 20
best_val_loss = float('inf')
early_stopping_patience = 5
early_stopping_counter = 0

for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()

    # Validation phase
    resnet.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

     # Save model after each epoch
    torch.save(resnet.state_dict(), f'resnet18_epoch_{epoch+1}.pth')

    # Check for improvement and early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
        torch.save(resnet.state_dict(), 'best_resnet18.pth')
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered")
            break

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(validation_loader)}")

Epoch 1/20, Training Loss: 4.015895710541652, Validation Loss: 3.656312007170457
Epoch 2/20, Training Loss: 2.216773120256571, Validation Loss: 2.6621320797846866
Epoch 3/20, Training Loss: 1.011869883308044, Validation Loss: 2.3262189076497006
Epoch 4/20, Training Loss: 0.3608705375630122, Validation Loss: 2.0284662521802463
Epoch 5/20, Training Loss: 0.11998395836697175, Validation Loss: 1.6336029034394484
Epoch 6/20, Training Loss: 0.03672697079869417, Validation Loss: 1.3854215695307806
Epoch 7/20, Training Loss: 0.013806459090958994, Validation Loss: 1.2854520907768836
Epoch 8/20, Training Loss: 0.005279537724653402, Validation Loss: 1.2576018846951997
Epoch 9/20, Training Loss: 0.004715515381226746, Validation Loss: 1.2594513572179353
Epoch 10/20, Training Loss: 0.004577144004332905, Validation Loss: 1.2526510724654565
Epoch 11/20, Training Loss: 0.004206859273836017, Validation Loss: 1.2456609698442311
Epoch 12/20, Training Loss: 0.003923040335603918, Validation Loss: 1.23296808

Test Dataset

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os, csv


class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [
            os.path.join(root_dir, img_name) for img_name in os.listdir(root_dir)
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return img_path, image


transform = transforms.Compose(
    [
        # transforms.Resize((299, 299)),  # Assuming Inception input size
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)



Inference

In [ ]:
from torchvision import datasets, transforms, models
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
import os, csv

# Define the number of classes
num_classes = 100
fourth_epoch_path="resnet_epoch_4.pth"
final_model_path="best_resnet18.pth"
# Load the saved state_dict
saved_state_dict = torch.load(final_model_path)

# Create a new ResNet model for testing with the correct number of classes
resnet_test = models.resnet18(pretrained=False)
resnet_test.fc = nn.Linear(resnet_test.fc.in_features, num_classes)

# Load the state_dict into the new testing model
resnet_test.load_state_dict(saved_state_dict)

# Set the model to evaluation mode
resnet_test.eval()
print("starting eval")
# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet_test.to(device)

# Define the transform for testing images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create the test dataset and data loader
test_dataset = TestDataset(root_dir="test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

predictions = []
print("performing inference")
# Perform inference
with torch.no_grad():
    for img_path, inputs in test_loader:
        inputs = inputs.to(device)

        # Forward pass
        outputs = resnet_test(inputs)

        # Get predicted labels
        _, predicted = torch.max(outputs, 1)

        # Store predictions
        for path, label in zip(img_path, predicted.cpu().numpy()):
            predictions.append({"id": path.replace("\\","/"), "Predicted": label})

# Save predictions to a CSV file
csv_file = "predictions.csv"
with open(csv_file, "w", newline="") as csvfile:
    fieldnames = ["id", "Predicted"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for prediction in predictions:
        writer.writerow(prediction)

print(f"Predictions saved to {csv_file}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


starting eval
performing inference
Predictions saved to predictions.csv
